In [30]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, MaxPool2D, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import classification_report, make_scorer, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
satelites_data = np.loadtxt('satelites_grises.csv', delimiter=',') # 128x128
satelites_X = satelites_data[:,1:].reshape(-1,128,128,1)
satelites_y = satelites_data[:,0]
#satelites_X_train, satelites_X_test, satelites_y_train, satelites_y_test = train_test_split(satelites_X, satelites_y, stratify=satelites_y)

In [3]:
emojis_data = np.loadtxt('emojis.txt') # 32x32
emojis_X = emojis_data[:,1:].reshape(-1,32,32,1)
emojis_y = emojis_data[:,0] - 1
#emoji_X_train, emoji_X_test, emoji_y_train, emoji_y_test = train_test_split(emojis_X, emojis_y, stratify=emojis_y)

In [4]:
mnist_train_data = pd.read_csv("fashion-mnist_train.csv") # 28x28
mnist_test_data = pd.read_csv('fashion-mnist_test.csv')

mnist_X_train = mnist_train_data.iloc[:,1:].to_numpy().reshape(-1,28,28,1) / 255
mnist_y_train = mnist_train_data.iloc[:,0]
mnist_X_test = mnist_test_data.iloc[:,1:].to_numpy().reshape(-1,28,28,1) / 255
mnist_y_test = mnist_test_data.iloc[:,0]


In [36]:
fold = StratifiedKFold(shuffle=True, random_state=8)
def scoring_fun(y_true, y_pred):
    reports.append(classification_report(y_true, y_pred, output_dict=True))
    return accuracy_score(y_true, y_pred)

In [80]:
def kfold_summary(scoring_reports):
    acc = np.array([])
    macro_prec = np.array([])
    macro_rec = np.array([])
    macro_f1 = np.array([])
    macro_supp = np.array([])
    weigh_prec = np.array([])
    weigh_rec = np.array([])
    weigh_f1 = np.array([])
    weigh_supp = np.array([])

    for report in scoring_reports:
        acc = np.append(acc, report['accuracy'])
        macro_prec = np.append(macro_prec, report['macro avg']['precision'])
        macro_rec = np.append(macro_rec, report['macro avg']['recall'] )
        macro_f1 = np.append(macro_f1, report['macro avg']['f1-score'])
        macro_supp = np.append(macro_supp, report['macro avg']['support'])
        weigh_prec = np.append(weigh_prec, report['weighted avg']['precision'])
        weigh_rec = np.append(weigh_rec, report['weighted avg']['recall'] )
        weigh_f1 = np.append(weigh_f1, report['weighted avg']['f1-score'])
        weigh_supp = np.append(weigh_supp, report['weighted avg']['support'])
    
    print('\t\t\tMean\t\t\tStDev')
    print(f'\tACCURACY:\t{acc.mean()}\t{acc.std()}\n')
    print(f'Macro\tPrecision:\t{macro_prec.mean()}\t{macro_prec.std()}')
    print(f'\tRecall:\t\t{macro_rec.mean()}\t{macro_rec.std()}')
    print(f'\tf1:\t\t{macro_f1.mean()}\t{macro_f1.std()}')
    print(f'\tsupport:\t\t{macro_supp.mean()}\n')
    print(f'Weight\tPrecision:\t{weigh_prec.mean()}\t{weigh_prec.std()}')
    print(f'\tRecall:\t\t{weigh_rec.mean()}\t{weigh_rec.std()}')
    print(f'\tf1:\t\t{weigh_f1.mean()}\t{weigh_f1.std()}')
    print(f'\tsupport:\t\t{weigh_supp.mean()}')


# emojis

In [37]:
def emojis_cnn():
    emojis_model = Sequential()
    emojis_model.add(Conv2D(32, (2,2), activation='relu', input_shape=(32,32,1)))
    emojis_model.add(MaxPool2D(3,3))
    emojis_model.add(Flatten())
    emojis_model.add(Dense(32, activation='relu'))
    emojis_model.add(Dense(5, activation='softmax'))

    emojis_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return emojis_model

emojis_clf = KerasClassifier(build_fn=emojis_cnn, epochs= 1, batch_size=32, verbose=1) # ajustar epochs

reports = []

acuracy = cross_validate(emojis_clf, X=emojis_X, y=emojis_y, cv=fold ,scoring=make_scorer(scoring_fun))
kfold_summary(reports)

C:\Users\Alan\AppData\Local\Temp/ipykernel_23208/2301743090.py:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  emojis_clf = KerasClassifier(build_fn=emojis_cnn, epochs= 1, batch_size=32, verbose=1) # ajustar epochs


16/16 [==============================] - 0s 2ms/step


In [35]:
def mnist_cnn():
    mnist_model = Sequential()
    mnist_model.add(Conv2D(32, (2,2), activation='relu', input_shape=(28,28,1)))
    mnist_model.add(MaxPool2D(3,3))
    mnist_model.add(Flatten())
    mnist_model.add(Dense(32, activation='relu'))
    mnist_model.add(Dense(10, activation='softmax'))

    mnist_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return mnist_model

mnist_clf = KerasClassifier(build_fn=mnist_cnn, epochs= 1, batch_size=32, verbose=1) # ajustar epochs

mnist_clf.fit(mnist_X_train, mnist_y_train)

mnist_pred = mnist_clf.predict(mnist_X_test)
print(classification_report(mnist_y_test, mnist_pred))

              precision    recall  f1-score   support

         0.0       0.59      0.84      0.70       101
         1.0       0.98      0.59      0.74       101
         2.0       0.99      0.98      0.99       101
         3.0       0.75      0.62      0.67        91
         4.0       0.75      0.86      0.80       100

    accuracy                           0.78       494
   macro avg       0.81      0.78      0.78       494
weighted avg       0.81      0.78      0.78       494

              precision    recall  f1-score   support

         0.0       0.65      0.74      0.69       100
         1.0       0.91      0.78      0.84       102
         2.0       0.99      1.00      1.00       101
         3.0       0.78      0.52      0.62        91
         4.0       0.70      0.92      0.80       100

    accuracy                           0.80       494
   macro avg       0.81      0.79      0.79       494
weighted avg       0.81      0.80      0.79       494

              precisio

In [ ]:
def satelite_cnn():
    satelite_model = Sequential()
    satelite_model.add(Conv2D(32, (2,2), activation='relu', input_shape=(128,128,1)))
    satelite_model.add(MaxPool2D(3,3))
    satelite_model.add(Conv2D(64, (2,2), activation='relu'))
    satelite_model.add(MaxPool2D(3,3))
    satelite_model.add(Flatten())
    satelite_model.add(Dense(64, activation='relu'))
    satelite_model.add(Dense(6, activation='softmax'))

    satelite_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return satelite_model

satelite_clf = KerasClassifier(build_fn=satelite_cnn, epochs= 1, batch_size=32, verbose=1) # ajustar epochs

reports = []

acuracy = cross_validate(satelite_clf, X=satelites_X, y=satelites_y, cv=fold ,scoring=make_scorer(scoring_fun))
kfold_summary(reports)